In [1]:
# %pip install --force-reinstall --no-deps torch==2.2.2 torchvision==0.17.2

In [2]:
# %pip install statsforecast mlforecast pandas polars pyarrow holidays lightgbm neuralforecast

In [9]:
import pandas as pd
from utils.preprocessing import load_daily_data, load_monthly_data
from utils.baseline import run_baseline_forecast_daily, run_baseline_forecast_monthly
from utils.statistical import run_statistical_forecast_daily, run_statistical_forecast_monthly
from utils.machine_learning import run_machine_learning_forecast_daily, run_machine_learning_forecast_monthly
from utils.deep_learning import run_deep_learning_forecast_daily, run_deep_learning_forecast_monthly

### Loading Data:

In [4]:
daily_train, daily_val, daily_test = load_daily_data(use_existing=True)
monthly_train, monthly_val, monthly_test = load_monthly_data(use_existing=True)

### Forecasts:

##### Baseline Forecast:

In [5]:
base_daily_val, base_daily_test = run_baseline_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
base_monthly_val, base_monthly_test = run_baseline_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Statistical Forecast:

In [6]:
statis_daily_val, statis_daily_test = run_statistical_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
statis_monthly_val, statis_monthly_test = run_statistical_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

Machine Learning Forecast:

In [7]:
ml_daily_val, ml_daily_test = run_machine_learning_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
ml_monthly_val, ml_monthly_test = run_machine_learning_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

Deep Learning Forecast:

In [ ]:
dl_daily_val, dl_daily_test = run_baseline_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
dl_monthly_val, dl_monthly_test = run_baseline_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 25
[LightGBM] [Info] Number of data points in the train set: 3653, number of used features: 3
[LightGBM] [Info] Start training from score 76.349956
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 25
[LightGBM] [Info] Number of data points in the train set: 4009, number of used features: 3
[LightGBM] [Info] Start training from score 78.372437
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 155